<a href="https://colab.research.google.com/github/aeleraqi/Text-Mining/blob/main/Text_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Insatalling dependencies

In [ ]:
!pip install pandas transformers torch arabert

In [ ]:
!pip install camel-tools

In [ ]:
!pip install arabert

In [ ]:
!pip install emoji==1.7.0

  Using cached emoji-1.7.0-py3-none-any.whl
  Attempting uninstall: emoji
    Found existing installation: emoji 1.4.2
    Uninstalling emoji-1.4.2:
      Successfully uninstalled emoji-1.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arabert 1.0.1 requires emoji==1.4.2, but you have emoji 1.7.0 which is incompatible.


In [ ]:
import pandas as pd
from transformers import pipeline
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from camel_tools.sentiment import SentimentAnalyzer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from transformers import BertTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [ ]:
# Download stopwords for Arabic if you haven't done so already
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Load sentiment analysis pipeline for Arabic
sentiment_model = pipeline('sentiment-analysis', model='aubmindlab/bert-base-arabertv02')
sa = SentimentAnalyzer("CAMeL-Lab/bert-base-arabic-camelbert-ca-sentiment")
model_name = "malmarjeh/mbert2mbert-arabic-text-summarization"
preprocessor = ArabertPreprocessor(model_name="")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Load the mT5 model and tokenizer for summarization
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summarization_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Hardware accelerator e.g. GPU is availab

##Dataset

In [ ]:
# Read the Excel file
df = pd.read_excel("/content/مناخ الاستثمار.xlsx")

In [ ]:
df.shape

(41, 1)

In [ ]:
df.head(3)

,user Comment
0,مصر تخطط لإقرار حوافز لخفض معدل الإنجاب، ضمن م...
1,الإسراع في الانتقال الي العاصمة الإدارية الجدي...
2,: في خمس خطوات.. يمكن لمصر أن تزيد الصادرات وت...


In [ ]:
df.loc[1, 'user Comment']

'الإسراع في الانتقال الي العاصمة الإدارية الجديدة.والمدن الجديدة ▪️عندما قامت مصر بتقديم ملفها لتنظيم كأس العالم 2010 تم رفض ملفها بالإجماع بسبب أن العاصمة المصرية كانت لا تصلح إطلاقا لتنظيم مناسبات كبرى بسبب أنها كانت مصابه بجلطات مرورية يستحيل علاجها إلا بمعجزة ▪️حتى أن أحد المستثمرين الاجانب عندما زار مصر سنة ٢٠١٣ و قابل كذا شركة مصرية ناشئة بهدف الأستثمار فيهم و اثناء قرب نهاية رحلته سأله المرافق المصري له ، ايه رأيك في الشركات اللي انت شفتها ؟ وهل هتستثمر في اي واحدة ؟ ** رد عليه الرجل "بالنسبة للسؤال الأول ، في شركات منهم ممتازة و تستحق الأستثمار" ، و بالنسبة للثاني "في ظل وجودهم في القاهرة مستحيل استثمر فيهم" ‼️ ** سأله المرافق المصري ليه كده؟ .. فكان الرد من المستثمر صادم له لكنه منطقي ، حيث قال : مدينتكم دي بأحتاج فيها ساعتين علشان اروح من أي نقطة لأي نقطة .. ده قاتل للأستثمار ، أصل مفيش حد حييجي يستثمر في مدينة الناس فيها بيقضوا نصف حياتهم في الشوارع !! ▪️حتى أن إحدي الدراسات الأجنبية وصلت لرقم مفرع بيقول ان تكلفة الأزدحام المروري في القاهرة الكبرى (القاهرة و الجيزة و القليوب

##Text Cleaning

In [ ]:
# Define the list of additional Arabic stopwords
extra_arabic_stopwords = [
    'و', 'هو', 'هي', 'أن', 'إن', 'التي', 'لو', 'لـ', 'وهو', 'وعلى', 'بينما',
    'بناءً عليه', 'وأن', 'وإن', 'ده', 'وده', 'الا','إلى','زي','او', 'إلا', 'علي', 'ان', 'وان',
    'علشان', 'حتى', 'دى', 'ل', 'وهي','التي', 'التي','الي', 'عندما', 'تم', 'ان', 'كانت',
    'حتي', 'احد', 'فيهم', 'ايه', 'رايك','أهم', 'اللي', 'انت', 'هل', 'في','فى', 'لي',
    'كده', 'فكان', 'لكن', 'قال', 'دي', 'لاي', 'حتي', 'ان', 'فقط', 'بمعني',
    'ولازم', 'يتم', 'علشان', 'كده', 'علي', 'ثم', 'قد','عشان','اهم', 'إلي', 'من', 'مع',
    'كما', 'بأن', 'أو', 'أم', 'ثم', 'إذا', 'إذ', 'على', 'عن', 'ذلك', 'بما',
    'لأن', 'إذ', 'حين', 'بعد', 'قبل', 'أي', 'أنها', 'هو', 'هي', 'ما', 'هذا'
]

# Combine default Arabic stopwords with the extra ones
stop_words = set(stopwords.words('arabic')).union(extra_arabic_stopwords)

# Function to remove stopwords
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

# Function to remove punctuation including ‼️ and **
def remove_punctuation(text):
    # Basic punctuation and symbols
    text = re.sub(r'[،؛؟\[\](){}<>.,;:"\'!?]', '', text)
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)

    # Additional specific symbols
    text = re.sub(r'[‼️**]+', '', text)

    return text.replace('؛', '').strip()

# Function to remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# Function to remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text).strip()

# Function to remove general emoticons using regex
def remove_emoticons(text):
    emoticon_pattern = re.compile(r'(:\)|:\(|:D|:P|;\)|:-\)|:-\(|:-D|:-P|:o)')
    return emoticon_pattern.sub(r'', text).strip()

# Function to convert Arabic numbers to English if not already in that format
def convert_numbers(text):
    arabic_to_english = str.maketrans('٠١٢٣٤٥٦٧٨٩', '0123456789')
    return text.translate(arabic_to_english)

# Function to clean text
def clean_text(text):
    text = remove_emoji(text)  # Remove emojis
    text = remove_punctuation(text)  # Remove punctuation including ‼️ and **
    text = remove_urls(text)  # Remove URLs
    text = remove_emoticons(text)  # Remove emoticons
    text = remove_stopwords(text)  # Remove stopwords
    text = convert_numbers(text)  # Convert Arabic numbers to English if not already
    return text

# Assuming 'df' is your DataFrame and 'user Comment' is the column to clean
df['clean Comment'] = df['user Comment'].apply(clean_text)

# Remove duplicated rows based on the 'clean Comment' column
df.drop_duplicates(subset=['clean Comment'], inplace=True)


In [ ]:
df.shape

(40, 2)

In [ ]:
df.loc[1, 'clean Comment']

'الإسراع الانتقال العاصمة الإدارية الجديدةوالمدن الجديدة قامت مصر بتقديم ملفها لتنظيم كأس العالم 2010 رفض ملفها بالإجماع بسبب العاصمة المصرية تصلح إطلاقا لتنظيم مناسبات كبرى بسبب مصابه بجلطات مرورية يستحيل علاجها بمعجزة المستثمرين الاجانب زار مصر سنة 2013 قابل شركة مصرية ناشئة بهدف الأستثمار اثناء قرب نهاية رحلته سأله المرافق المصري رأيك الشركات شفتها وهل هتستثمر اي واحدة رد الرجل بالنسبة للسؤال الأول شركات منهم ممتازة تستحق الأستثمار بالنسبة للثاني ظل وجودهم القاهرة مستحيل استثمر سأله المرافق المصري ليه الرد المستثمر صادم لكنه منطقي مدينتكم بأحتاج ساعتين اروح نقطة لأي نقطة قاتل للأستثمار أصل مفيش حد حييجي يستثمر مدينة الناس بيقضوا نصف حياتهم الشوارع إحدي الدراسات الأجنبية وصلت لرقم مفرع بيقول تكلفة الأزدحام المروري القاهرة الكبرى القاهرة الجيزة القليوبية بلغ 65 مليار رقم اكبر دخل قناة السويس وأكثر حجم الإستثمار الأجنبي المباشر وصل مصر بالعام الماضي القاهرة تعد تصلح نهائيا كعاصمة تهجير نصف سكانها الإنتقال لعاصمة أخرى لجأت مصر للحل الثاني عمل عاصمة جديدة أحدث طراز عالمي'

##Sentiment

In [ ]:
!pip install keras-preprocessing

In [ ]:
!pip install keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example setup for LSTM model
max_words = 5000
max_len = 150
embedding_dim = 100

# Tokenize and pad the text data
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(df['clean Comment'].values)
X = tokenizer.texts_to_sequences(df['clean Comment'].values)
X = pad_sequences(X, maxlen=max_len)

# Build a simple LSTM model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))  # 3 classes: Positive, Negative, Neutral

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the sentiment labels
sentiment_labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}

# Function to predict sentiment using the trained LSTM model and return labels
def get_sentiment_lstm(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(padded)
    sentiment_idx = np.argmax(pred, axis=1)[0]
    return sentiment_labels[sentiment_idx]

# Apply sentiment analysis using LSTM to the 'clean Comment' column
df['sentiment'] = df['clean Comment'].apply(get_sentiment_lstm)

# Apply sentiment analysis using LSTM to the original 'user Comment' column
df['sentiment1'] = df['user Comment'].apply(get_sentiment_lstm)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [ ]:
df.head(2)

,user Comment,clean Comment,sentiment,sentiment1
0,مصر تخطط لإقرار حوافز لخفض معدل الإنجاب، ضمن م...,مصر تخطط لإقرار حوافز لخفض معدل الإنجاب ضمن مش...,Positive,Positive
1,الإسراع في الانتقال الي العاصمة الإدارية الجدي...,الإسراع الانتقال العاصمة الإدارية الجديدةوالمد...,Neutral,Neutral


In [ ]:
import plotly.graph_objects as go

# Count the number of comments per sentiment for 'clean Comment'
sentiment_counts_clean = df['sentiment'].value_counts()

# Count the number of comments per sentiment for 'user Comment'
sentiment_counts_user = df['sentiment1'].value_counts()

# Define the colors for each sentiment
colors = {'Negative': 'red', 'Neutral': 'orange', 'Positive': 'green'}

# Create the bar chart for 'clean Comment'
fig_clean = go.Figure(data=[
    go.Bar(
        x=sentiment_counts_clean.index,
        y=sentiment_counts_clean.values,
        marker_color=[colors[sentiment] for sentiment in sentiment_counts_clean.index]
    )
])

fig_clean.update_layout(
    title='Number of Comments by Sentiment (Clean Comment)',
    xaxis_title='Sentiment',
    yaxis_title='Number of Comments'
)

# Show the chart
fig_clean.show()

# Create the bar chart for 'user Comment'
fig_user = go.Figure(data=[
    go.Bar(
        x=sentiment_counts_user.index,
        y=sentiment_counts_user.values,
        marker_color=[colors[sentiment] for sentiment in sentiment_counts_user.index]
    )
])

fig_user.update_layout(
    title='Number of Comments by Sentiment (User Comment)',
    xaxis_title='Sentiment',
    yaxis_title='Number of Comments'
)

# Show the chart
fig_user.show()


In [ ]:
from collections import Counter

def analyze_text(text):
    # Tokenize the text
    words = text.split()

    # Count word frequencies
    word_freq = Counter(words)

    # Filter out words with a frequency of less than 2
    filtered_word_freq = {word: count for word, count in word_freq.items() if count >= 2}

    # Get the top 5 most frequent words with their frequencies
    most_frequent_words = sorted(filtered_word_freq.items(), key=lambda x: x[1], reverse=True)[:5]

    return most_frequent_words

# Apply the analysis function to each row and store the results in a new column
df['Most Frequent Words with Frequencies'] = df['clean Comment'].apply(analyze_text)

In [ ]:
df.head(5)

,user Comment,clean Comment,sentiment,sentiment1,Most Frequent Words,Most Frequent Words with Frequencies
0,مصر تخطط لإقرار حوافز لخفض معدل الإنجاب، ضمن م...,مصر تخطط لإقرار حوافز لخفض معدل الإنجاب ضمن مش...,Positive,Positive,"[الإنجاب, طفل, مصر, تخطط, لإقرار]","[(الإنجاب, 2), (طفل, 2)]"
1,الإسراع في الانتقال الي العاصمة الإدارية الجدي...,الإسراع الانتقال العاصمة الإدارية الجديدةوالمد...,Neutral,Neutral,"[مصر, القاهرة, العاصمة, ملفها, لتنظيم]","[(مصر, 4), (القاهرة, 4), (العاصمة, 2), (ملفها,..."
2,: في خمس خطوات.. يمكن لمصر أن تزيد الصادرات وت...,خطوات يمكن لمصر تزيد الصادرات وتجذب المزيد الا...,Neutral,Neutral,"[مليار, الاستثمار, الأجنبي, –, الصناعات]","[(مليار, 5), (الاستثمار, 4), (الأجنبي, 4), (–,..."
3,خطيئة الصادرات التقليدية ▪️هل تعلم يا صديقي ان...,خطيئة الصادرات التقليدية تعلم صديقي مصر تقوم ب...,Positive,Positive,"[البطاطس, يصل, مليار, تعلم, مليون]","[(البطاطس, 7), (يصل, 5), (مليار, 5), (تعلم, 4)..."
4,"موقع ""ناشيونال إنترست"" في تقرير صحفي بعنوان "" ...",موقع ناشيونال إنترست تقرير صحفي بعنوان ⁧‫ السع...,Neutral,Neutral,"[⁧‫, بقيادة, موقع, ناشيونال, إنترست]","[(⁧‫, 2), (بقيادة, 2)]"


In [ ]:
# Initialize lists to hold the words and frequencies
words = []
frequencies = []

# Loop through each row in the DataFrame and extract words and frequencies
for row in df['Most Frequent Words with Frequencies']:
    for word, freq in row:
        words.append(word)
        frequencies.append(freq)

# Create a DataFrame for plotting
plot_df = pd.DataFrame({'Word': words, 'Frequency': frequencies})

# Aggregate frequencies by summing up the counts for duplicate words
plot_df = plot_df.groupby('Word').sum().reset_index()

# Sort the DataFrame by frequency in descending order
plot_df = plot_df.sort_values(by='Frequency', ascending=False)

# Plotting the bar chart
fig = go.Figure(data=[go.Bar(
    x=plot_df['Word'],
    y=plot_df['Frequency'],
    marker_color='blue'  # You can choose any color
)])

fig.update_layout(
    title='Most Frequent Words with Frequencies',
    xaxis_title='Words',
    yaxis_title='Frequency',
    xaxis_tickangle=-45  # Rotate x-axis labels for better readability
)

# Show the chart
fig.show()

##Summaries

In [ ]:
from transformers import MBartForConditionalGeneration, MBartTokenizer
import nltk

# Initialize the tokenizer and model for Arabic summarization
model_name = "facebook/mbart-large-50"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)
summarization_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer)

# Function to enhance the summary into bullet points
def enhance_summary_to_bullets(summary):
    # Split the summary into sentences
    sentences = nltk.sent_tokenize(summary)
    # Convert sentences to bullet points
    bullets = "\n".join([f"- {sentence}" for sentence in sentences])
    return bullets

# Create lists to store the summaries and bullet points
summaries = []
bullet_points_list = []

# Maximum length for the summary
max_summary_length = 250

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    text = row['user Comment']  # Access the 'user Comment' for the current row
    text = preprocessor.preprocess(text)

    if len(text) < max_summary_length:
        # If text is shorter than 250 characters, use the text as summary
        summary = text
    else:
        # Split the text into chunks of maximum sequence length
        max_length = 1024  # Adjust this value based on your model's maximum sequence length
        chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]

        results = []

        # Process each chunk separately
        for chunk in chunks:
            # Use the summarization pipeline
            summary_chunk = summarization_pipeline(chunk, max_length=200, min_length=30, do_sample=False)
            result = summary_chunk[0]['summary_text']  # Extract the summary text
            results.append(result)

        # Combine the results into a single summary for this row
        summary = " ".join(results)

        # Trim the summary to the maximum length of 250 characters
        summary = summary[:max_summary_length]

    # Add the summary to the list
    summaries.append(summary)

    # Enhance the summary into bullet points
    bullet_points = enhance_summary_to_bullets(summary)
    bullet_points_list.append(bullet_points)

# Add the summaries and bullet points to the DataFrame in new columns
df['Summary'] = summaries
df['Bullet Points'] = bullet_points_list


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 200, but your input_length is only 88. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your max_length is set to 200, but your input_length is only 82. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your max_length is set to 200, but your input_length is only 108. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)
Your max_length is set to 200, but your input_length is only 77. Since this is a summarization task, where

In [ ]:
df.head(2)

,user Comment,clean Comment,sentiment,sentiment1,Most Frequent Words,Most Frequent Words with Frequencies,Summary,Bullet Points
0,مصر تخطط لإقرار حوافز لخفض معدل الإنجاب، ضمن م...,مصر تخطط لإقرار حوافز لخفض معدل الإنجاب ضمن مش...,Positive,Positive,"[الإنجاب, طفل, مصر, تخطط, لإقرار]","[(الإنجاب, 2), (طفل, 2)]",تخطط لإقرار حوافز لخفض معدل الإنجاب ، ضمن مشرو...,- تخطط لإقرار حوافز لخفض معدل الإنجاب ، ضمن مش...
1,الإسراع في الانتقال الي العاصمة الإدارية الجدي...,الإسراع الانتقال العاصمة الإدارية الجديدةوالمد...,Neutral,Neutral,"[مصر, القاهرة, العاصمة, ملفها, لتنظيم]","[(مصر, 4), (القاهرة, 4), (العاصمة, 2), (ملفها,...",الإسراع في الانتقال الي العاصمة الإدارية الجدي...,- الإسراع في الانتقال الي العاصمة الإدارية الج...


In [ ]:
df.loc[8, 'Summary']

'بسولة اقتصاد : الازمة ليست ازمة دولار ، ... ... بل ازمة جنيه ! ! موارد مصر الدولارية من التصدير وقناة السويس وتحويلات المصريين والاستثمار الاجنبى المباشر والسياحة بلغت عام 2022 حوالى 120 مليار دولار ( 53 + 8 + 33 + 12 + 13 مليار على الترتيب ) وهو ماي'

In [ ]:
df.loc[8, 'Bullet Points']

'- بسولة اقتصاد : الازمة ليست ازمة دولار ، ... ... بل ازمة جنيه !\n- !\n- موارد مصر الدولارية من التصدير وقناة السويس وتحويلات المصريين والاستثمار الاجنبى المباشر والسياحة بلغت عام 2022 حوالى 120 مليار دولار ( 53 + 8 + 33 + 12 + 13 مليار على الترتيب ) وهو ماي'

**NER**

In [ ]:
from transformers import pipeline

# Load the NER model
ner_pipeline = pipeline("ner", model="marefa-nlp/marefa-ner", tokenizer="marefa-nlp/marefa-ner")

# Function to apply NER
def extract_entities(text):
    entities = ner_pipeline(text)
    return [(entity['word'], entity['entity']) for entity in entities]

# Apply NER to 'clean Comment'
df['Entities'] = df['clean Comment'].apply(extract_entities)


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
df.head(10)

,user Comment,clean Comment,sentiment,sentiment1,Most Frequent Words,Most Frequent Words with Frequencies,Summary,Bullet Points,Entities
0,مصر تخطط لإقرار حوافز لخفض معدل الإنجاب، ضمن م...,مصر تخطط لإقرار حوافز لخفض معدل الإنجاب ضمن مش...,Positive,Positive,"[الإنجاب, طفل, مصر, تخطط, لإقرار]","[(الإنجاب, 2), (طفل, 2)]",تخطط لإقرار حوافز لخفض معدل الإنجاب ، ضمن مشرو...,- تخطط لإقرار حوافز لخفض معدل الإنجاب ، ضمن مش...,[]
1,الإسراع في الانتقال الي العاصمة الإدارية الجدي...,الإسراع الانتقال العاصمة الإدارية الجديدةوالمد...,Neutral,Neutral,"[مصر, القاهرة, العاصمة, ملفها, لتنظيم]","[(مصر, 4), (القاهرة, 4), (العاصمة, 2), (ملفها,...",الإسراع في الانتقال الي العاصمة الإدارية الجدي...,- الإسراع في الانتقال الي العاصمة الإدارية الج...,"[(▁كأس, B-event), (▁العالم, I-event), (▁2010, ..."
2,: في خمس خطوات.. يمكن لمصر أن تزيد الصادرات وت...,خطوات يمكن لمصر تزيد الصادرات وتجذب المزيد الا...,Neutral,Neutral,"[مليار, الاستثمار, الأجنبي, –, الصناعات]","[(مليار, 5), (الاستثمار, 4), (الأجنبي, 4), (–,...",في خمس خطوات . . يمكن لمصر أن تزيد الصادرات وت...,- في خمس خطوات .\n- .\n- يمكن لمصر أن تزيد الص...,"[(▁اليوم, I-time), (▁صندوق, B-organization), (..."
3,خطيئة الصادرات التقليدية ▪️هل تعلم يا صديقي ان...,خطيئة الصادرات التقليدية تعلم صديقي مصر تقوم ب...,Positive,Positive,"[البطاطس, يصل, مليار, تعلم, مليون]","[(البطاطس, 7), (يصل, 5), (مليار, 5), (تعلم, 4)...",يئة الصادرات التقليدية هل تعلم يا صديقي ان مصر...,- يئة الصادرات التقليدية هل تعلم يا صديقي ان م...,"[(▁مصر, B-location), (▁أوروبا, B-location), (▁..."
4,"موقع ""ناشيونال إنترست"" في تقرير صحفي بعنوان "" ...",موقع ناشيونال إنترست تقرير صحفي بعنوان ⁧‫ السع...,Neutral,Neutral,"[⁧‫, بقيادة, موقع, ناشيونال, إنترست]","[(⁧‫, 2), (بقيادة, 2)]",""" ناشيونال إنترست "" في تقرير صحفي بعنوان "" # ا...","- "" ناشيونال إنترست "" في تقرير صحفي بعنوان "" #...","[(▁إن, I-organization), (▁الولايات, B-location..."
5,السعودية عليت بمستوي المنافسة فوق خالص . التسه...,السعودية عليت بمستوي المنافسة خالص التسهيلات ا...,Negative,Negative,"[السعودية, …, إعفاء, تعليق, بنسبة]","[(السعودية, 2), (…, 2), (إعفاء, 2), (تعليق, 2)...",عليت بمستوي المنافسة فوق خالص . التسهيلات المم...,- عليت بمستوي المنافسة فوق خالص .\n- التسهيلات...,"[(▁أ, B-organization), (بل, B-organization), (..."
6,علي مساحة 3 مليون متر مربع. اختارت آبل العاصمة...,مساحة 3 مليون متر مربع اختارت آبل العاصمة السع...,Neutral,Neutral,"[مساحة, 3, مليون, متر, مربع]",[],مساحة 3 مليون متر مربع . اختارت آبل العاصمة ال...,- مساحة 3 مليون متر مربع .\n- اختارت آبل العاص...,"[(▁منطقة, B-location), (▁الشرق, I-location), (..."
7,شركة آبل أنهت المفاوضات مع المملكة السعودية عل...,شركة آبل أنهت المفاوضات المملكة السعودية تكون ...,Positive,Positive,"[شركة, المملكة, السعودية, تكون, مساحة]","[(شركة, 2), (المملكة, 2), (السعودية, 2), (تكون...",آبل أنهت المفاوضات مع المملكة السعودية علشان ت...,- آبل أنهت المفاوضات مع المملكة السعودية علشان...,"[(▁آب, I-organization), (ل, I-organization), (..."
8,كبسولة اقتصاد : الازمة ليست ازمة دولار ،…… بل ...,كبسولة اقتصاد الازمة ازمة …… ازمة موارد مصر ال...,Negative,Negative,"[الموازنة, اقتصاد, عام, مليار, حجم]","[(الموازنة, 4), (اقتصاد, 3), (عام, 3), (مليار,...",بسولة اقتصاد : الازمة ليست ازمة دولار ، ... .....,- بسولة اقتصاد : الازمة ليست ازمة دولار ، ... ...,"[(▁2022, B-time), (▁مصر, B-location), (▁ها, B-..."
9,الأزمة الاقتصادية الحالية في مصر تستلزم الآتي ...,الأزمة الاقتصادية الحالية مصر تستلزم الآتي وجه...,Neutral,Neutral,"[اعمل, التراخيص, برة, القطاع, الخاص]","[(اعمل, 5), (التراخيص, 3), (برة, 3), (القطاع, ...",الاقتصادية الحالية في مصر تستلزم الآتي من وجهة...,- الاقتصادية الحالية في مصر تستلزم الآتي من وج...,"[(▁الأزمة, B-event), (▁الاقتصادية, I-event), (..."


In [ ]:
df.to_excel("1مناخ الاستثمار.xlsx")